# DEMO
## Illustrating MIA Attack on SNNs and ANNs

In [1]:
import sys
import torch
import pickle
import numpy as np

sys.path.append("../")
from utils import *
from torch.utils.data import Subset
from Preprocess import get_dataloader_from_dataset, load_dataset
from Attacks.utils import *
from Attacks import *

import warnings
warnings.filterwarnings("ignore")



ModuleNotFoundError: No module named 'torch'

## Initialize Experiment Configuration

In [2]:
ATTACK_TYPE = "rmia" # ["attack_p", "attack_r", "rmia"]
DATASET_NAME = "cifar100" # ["cifar10", "cifar100", "imagenette", "imagewoof"]
# VGG16 is not trained for imagenet based datasets
MODEL_NAME = "resnet18" # ["vgg16", "resnet18", "resnet34"]
#MODEL_TYPE = "ann" # ["ann", "snn"]
MODEL_TYPE = {
    "model_0": "ann", 
    "model_1": "ann",
    "model_2": "ann",
    "model_3": "ann",
    "model_4": "ann"
}
# If model type is ann then n_steps has no significance
N_STEPS = 1
CHECKPOINT = './saved_models'
REFERENCE_MODELS = 4
CALIBRATION = False
# If calibration is False then dropout and n_samples are not considered
DROPOUT = 0.01 
N_SAMPLES = 10
# general variables
batch_size = 64
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NameError: name 'torch' is not defined

## Initializing relevant libraries

In [2]:
# Creating directory to save trained models and their logs
primary_model_path = os.path.join(CHECKPOINT, DATASET_NAME, MODEL_NAME, f"ref_models_{REFERENCE_MODELS}")

## Data Prep

#### Load Dataset

In [ ]:
dataset = load_dataset(DATASET_NAME, logger)
try:
    data_split_file = os.path.join(primary_model_path, "data_splits.pkl")
    with open(data_split_file, 'rb') as file:
        data_split_info = pickle.load(file)
    print("Data split information successfully loaded:")
except FileNotFoundError:
    print(f"Error: The file '{data_split_file}' does not exist")
# Creating dataloader
train_idxs = data_split_info[0]["train"]
test_idxs = data_split_info[0]["test"]
print(f"Dataset Specs : Train size {len(train_idxs)}, Test size {len(test_idxs)}")

#### Creating Dataloader

In [ ]:
data_loader = get_dataloader_from_dataset(
    DATASET_NAME, 
    Subset(dataset, np.concatenate((train_idxs, test_idxs), axis=0)), 
    batch_size=batch_size, 
    train=False
)

## Load Model

In [ ]:
target_model, reference_models = load_model(
    MODEL_NAME, DATASET_NAME, MODEL_TYPE, REFERENCE_MODELS, primary_model_path, device, N_STEPS
)

## Perform Attack

#### Initiliaze attack

In [ ]:
attack = perform_MIA(
            attack_type = ATTACK_TYPE, 
            model_type = MODEL_TYPE,
            target_model = target_model, 
            reference_models = reference_models, 
            data_loader = data_loader, 
            device = device,
            n_steps = N_STEPS,
            calibration = CALIBRATION,
            dropout = DROPOUT,
            n_samples = N_SAMPLES,
        )

#### Get Results

In [ ]:
attack.compute_scores()
scores = attack.scores
attack.get_results()
results = attack.results

### Visualize

##### ROC Curve

In [ ]:
plot_auc(results)

#### Distribution of MIA Scores

In [ ]:
plot_histogram(scores)